In [91]:
from mgmt.data.pickle import load_subject_pickles
import torch
import sklearn

%load_ext autoreload
%autoreload 2

from collections import defaultdict

from sklearn.model_selection import StratifiedKFold, KFold
import numpy as np
import math
from loguru import logger

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
subjects = load_subject_pickles(
    "/home/bryan/expr/brain_tumor/preprocess-subjects-v2/resample-2.0-crop-64",
)

loading subjects from pickles: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 653/653 [00:03<00:00, 164.73it/s]


In [5]:
generator = torch.Generator().manual_seed(1)

In [ ]:
train_test_val_ratio = (0.60, 0.15, 0.25)

In [ ]:
train_subjects, val_subjects = subjects_train_val_split(
    subjects, self.cfg.DATA.TRAIN_VAL_RATIO, generator)

In [16]:
class_counts = defaultdict(int)
for subject in subjects:
    if subject.train_test_split == "train":
        class_counts[subject.category] += 1

In [17]:
class_counts

defaultdict(int, {'methylated': 301, 'unmethylated': 276})

In [20]:
train_subjects = [x for x in subjects if x.train_test_split == "train"]

In [24]:
indices = np.arange(len(train_subjects))
category_ids = np.array([x.category_id for x in train_subjects])

In [106]:
np.random.seed(1)
skf = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 10)

In [108]:
np.random.seed(2)
skf2 = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 10)

In [33]:
# for i, (train_index, test_index) in enumerate(skf.split(indices, category_ids)):
#     print(f"Fold {i}:")
#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")

In [107]:
train_fold, test_fold = next(skf.split(indices, category_ids))

In [109]:
train_fold2, test_fold2 = next(skf2.split(indices, category_ids))

In [48]:
category_ids[train_fold]

array([1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,

In [57]:
def get_class_counts(selected_indices, category_ids):    
    sel_category_ids = category_ids[selected_indices]
    num_methyl = np.sum(sel_category_ids)
    num_unmethyl = len(sel_category_ids) - num_methyl
    methyl_perc = 100 * num_methyl / len(sel_category_ids)
    print(f"Methylated: {num_methyl} ({methyl_perc:.2f}%), unmethylated: {num_unmethyl}")

In [101]:
len(category_ids)

577

In [58]:
get_class_counts(np.arange(len(category_ids)), category_ids)

Methylated: 301 (52.17%), unmethylated: 276


In [56]:
get_class_counts(train_fold, category_ids)

Methylated: 200 (52.08%), unmethylated: 184


In [59]:
# wrapper around kfold where I specify the number of percentage of dataset
# for validation set, -> then determine the number of folds -> 
# then select the fold index

In [60]:
skf.split(indices, category_ids)

<generator object _BaseKFold.split at 0x7f3033c49d20>

In [98]:
def kfold_train_val_split(subjects, val_ratio = 0.20, fold_index = 0, random_state = 1):
    k_folds = math.floor(1 / val_ratio)
    if fold_index > k_folds:
        logger.warning(f"Kfold index({fold_index}) > # Folds ({k_folds})")
    fold_index = fold_index % k_folds
    skf = StratifiedKFold(n_splits = k_folds, shuffle = True, random_state = 1)
    indices = np.arange(len(subjects))
    category_ids = np.array([x.category_id for x in subjects])
    skf_splits = list(skf.split(indices, category_ids))
    train_indices, val_indices = skf_splits[fold_index]
    train_subjects = [subjects[i] for i in train_indices]
    val_subjects = [subjects[i] for i in val_indices]
    return train_subjects, val_subjects

In [99]:
train_sub, val_sub = kfold_train_val_split(train_subjects)

In [103]:
indices = np.arange(len(train_subjects))
category_ids = np.array([x.category_id for x in train_subjects])

In [104]:
np.random.seed(10)

In [111]:

indices = np.arange(10)
np.random.shuffle(indices)

In [113]:
np.random.shuffle?

In [116]:
subjects[0].get("train_test_split")

'train'

In [119]:
N = 500
K = 425
0.2 * N / K

0.23529411764705882

In [120]:
368 / 577

0.6377816291161178

In [121]:
86 / 577

0.14904679376083188

In [122]:
121 / 577

0.2097053726169844